# 🏎️ Shell Eco-marathon: Live Racing Line System V7

## Complete System with Location Sync, Streaming & Smooth Ideal Line

This notebook covers:
1. **Location-Based Sync** - Camera & Joulemeter matched by GPS position
2. **Live Streaming** - Real-time data reception and processing
3. **Ideal Line Creation** - How we build the optimal racing path
4. **Smoothing Algorithms** - Maximum accuracy & smooth visualization
5. **Testing & Validation** - Built-in tests for smoothness and accuracy

---

## 📚 Understanding the Ideal Racing Line

### What is the Ideal Line?

The **ideal racing line** is a GPS path that represents the most energy-efficient way to complete the track. We build it by:

1. **Dividing each lap into segments** (Q1, Q2, Q3, Q4)
2. **Measuring efficiency** (km/kWh) for each segment of each lap
3. **Selecting the best segment** from ALL laps for each quarter
4. **Combining them** into one "ideal" lap that may never have actually been driven!

```
Lap 1:  [Q1: 130 km/kWh] [Q2: 145 km/kWh] [Q3: 120 km/kWh] [Q4: 140 km/kWh]
Lap 2:  [Q1: 150 km/kWh] [Q2: 135 km/kWh] [Q3: 155 km/kWh] [Q4: 130 km/kWh]
Lap 3:  [Q1: 140 km/kWh] [Q2: 160 km/kWh] [Q3: 145 km/kWh] [Q4: 165 km/kWh]
                ↓               ↓               ↓               ↓
IDEAL:  [Q1: Lap 2 ✓ ] [Q2: Lap 3 ✓ ] [Q3: Lap 2 ✓ ] [Q4: Lap 3 ✓ ]
        = 150 + 160 + 155 + 165 = BEST POSSIBLE EFFICIENCY!
```

---

## Part 1: Setup & Dependencies

In [ ]:
# Install dependencies
!pip install pandas numpy matplotlib folium scipy flask-ngrok pyngrok opencv-python-headless -q
!pip install scikit-learn -q
print("✅ Dependencies installed!")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrowPatch
from math import radians, sin, cos, sqrt, atan2, degrees
import folium
from folium import plugins
import json
import time
import threading
from collections import deque, OrderedDict
from datetime import datetime
from scipy import interpolate
from scipy.ndimage import gaussian_filter1d
from scipy.signal import savgol_filter
import warnings
warnings.filterwarnings('ignore')

# For Colab
try:
    from google.colab import files, output
    IN_COLAB = True
except:
    IN_COLAB = False

print("✅ Libraries imported!")
print(f"   Running in Colab: {IN_COLAB}")

## Part 2: Core GPS Functions

These functions handle all GPS calculations - distance, bearing, and coordinate math.

In [ ]:
# ============================================
# 🌍 GPS CALCULATION FUNCTIONS
# ============================================

def haversine_km(lat1, lon1, lat2, lon2):
    """
    Calculate the great-circle distance between two GPS points.
    
    The Haversine formula accounts for Earth's curvature, giving us
    accurate distances even over long ranges.
    
    Returns: Distance in kilometers
    """
    R = 6371  # Earth's radius in kilometers
    
    # Convert to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    # Differences
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Haversine formula
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    
    return R * c


def haversine_meters(lat1, lon1, lat2, lon2):
    """Distance in meters - more intuitive for short distances."""
    return haversine_km(lat1, lon1, lat2, lon2) * 1000


def calculate_bearing(lat1, lon1, lat2, lon2):
    """
    Calculate the compass bearing from point 1 to point 2.
    
    This tells us which direction we're heading:
    - 0° = North
    - 90° = East  
    - 180° = South
    - 270° = West
    
    Returns: Bearing in degrees (0-360)
    """
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    dlon = lon2 - lon1
    
    x = sin(dlon) * cos(lat2)
    y = cos(lat1) * sin(lat2) - sin(lat1) * cos(lat2) * cos(dlon)
    
    bearing = degrees(atan2(x, y))
    
    # Normalize to 0-360
    return (bearing + 360) % 360


def offset_point(lat, lon, bearing_deg, distance_m):
    """
    Calculate a new GPS point given a start point, bearing, and distance.
    
    Useful for:
    - Projecting points ahead of the car
    - Creating offset paths (left/right of ideal line)
    
    Returns: (new_lat, new_lon)
    """
    R = 6371000  # Earth's radius in meters
    
    lat1 = radians(lat)
    lon1 = radians(lon)
    bearing = radians(bearing_deg)
    
    lat2 = np.arcsin(
        sin(lat1) * cos(distance_m / R) +
        cos(lat1) * sin(distance_m / R) * cos(bearing)
    )
    
    lon2 = lon1 + atan2(
        sin(bearing) * sin(distance_m / R) * cos(lat1),
        cos(distance_m / R) - sin(lat1) * sin(lat2)
    )
    
    return degrees(lat2), degrees(lon2)


def cumulative_distance(coords):
    """
    Calculate cumulative distance along a path of coordinates.
    
    Input: List of [lat, lon] pairs
    Returns: Array of cumulative distances in km
    """
    distances = [0]
    for i in range(1, len(coords)):
        d = haversine_km(
            coords[i-1][0], coords[i-1][1],
            coords[i][0], coords[i][1]
        )
        distances.append(distances[-1] + d)
    return np.array(distances)


print("✅ GPS functions defined!")

# Quick test
test_dist = haversine_meters(24.734833, 46.699108, 24.735000, 46.699200)
print(f"   Test distance: {test_dist:.1f} meters")

## Part 3: Data Storage & Location-Based Sync

### How Location Sync Works

Since the camera and joulemeter are **separate systems** that may start at different times, we can't sync by timestamp. Instead, we sync by **GPS location**:

```
Camera System (starts 2:00 PM):       Joulemeter System (starts 2:05 PM):
┌─────────────────────────┐           ┌─────────────────────────┐
│ 📷 + 📍 GPS             │           │ ⚡ + 📍 GPS             │
│                         │           │                         │
│ Sends: lat, lon, frame  │           │ Sends: lat, lon, energy │
└───────────┬─────────────┘           └───────────┬─────────────┘
            │                                     │
            └──────────────┬──────────────────────┘
                           ▼
                  ┌─────────────────┐
                  │  CLOUD SERVER   │
                  │                 │
                  │ "Both at same   │
                  │  location?"     │
                  │                 │
                  │ Distance < 5m?  │
                  │      ↓          │
                  │   SYNC! ✅      │
                  └─────────────────┘
```

In [ ]:
# ============================================
# 📊 DATA STORAGE WITH LOCATION SYNC
# ============================================

class LocationSyncedDataStore:
    """
    Stores and syncs data from two separate GPS-equipped systems.
    
    The key insight: Both systems have GPS, so when they report
    the same location (within threshold), we know they're measuring
    the same moment - even if their clocks differ!
    """
    
    def __init__(self, sync_threshold_meters=5.0):
        # Separate buffers for each data source
        # We keep recent readings and try to match them by location
        self.camera_buffer = deque(maxlen=500)
        self.joulemeter_buffer = deque(maxlen=500)
        
        # Successfully synced telemetry points
        self.synced_telemetry = []
        
        # How close (in meters) two points must be to sync
        self.sync_threshold_m = sync_threshold_meters
        
        # Statistics
        self.stats = {
            'camera_received': 0,
            'joulemeter_received': 0,
            'synced_count': 0,
            'sync_distances': []  # Track how close our syncs are
        }
    
    def add_camera_reading(self, lat, lon, speed=None, frame_id=None):
        """
        Add a reading from the camera system.
        Immediately tries to find a matching joulemeter reading.
        """
        reading = {
            'latitude': lat,
            'longitude': lon,
            'speed': speed,
            'frame_id': frame_id,
            'timestamp': time.time(),
            'synced': False
        }
        self.camera_buffer.append(reading)
        self.stats['camera_received'] += 1
        
        # Try to sync with joulemeter
        return self._try_sync_camera(reading)
    
    def add_joulemeter_reading(self, lat, lon, voltage, current, power, energy_wh, speed=None):
        """
        Add a reading from the joulemeter system.
        Immediately tries to find a matching camera reading.
        """
        reading = {
            'latitude': lat,
            'longitude': lon,
            'voltage': voltage,
            'current': current,
            'power': power,
            'energy_wh': energy_wh,
            'speed': speed,
            'timestamp': time.time(),
            'synced': False
        }
        self.joulemeter_buffer.append(reading)
        self.stats['joulemeter_received'] += 1
        
        # Try to sync with camera
        return self._try_sync_joulemeter(reading)
    
    def _try_sync_camera(self, camera_reading):
        """
        Find the closest joulemeter reading to this camera position.
        """
        cam_lat = camera_reading['latitude']
        cam_lon = camera_reading['longitude']
        
        best_match = None
        best_distance = float('inf')
        best_idx = -1
        
        for idx, jm in enumerate(self.joulemeter_buffer):
            if jm['synced']:
                continue
            
            dist = haversine_meters(cam_lat, cam_lon, jm['latitude'], jm['longitude'])
            
            if dist < best_distance:
                best_distance = dist
                best_match = jm
                best_idx = idx
        
        # Check if within threshold
        if best_match and best_distance <= self.sync_threshold_m:
            return self._create_synced_point(camera_reading, best_match, best_distance)
        
        return None
    
    def _try_sync_joulemeter(self, jm_reading):
        """
        Find the closest camera reading to this joulemeter position.
        """
        jm_lat = jm_reading['latitude']
        jm_lon = jm_reading['longitude']
        
        best_match = None
        best_distance = float('inf')
        
        for cam in self.camera_buffer:
            if cam['synced']:
                continue
            
            dist = haversine_meters(jm_lat, jm_lon, cam['latitude'], cam['longitude'])
            
            if dist < best_distance:
                best_distance = dist
                best_match = cam
        
        if best_match and best_distance <= self.sync_threshold_m:
            return self._create_synced_point(best_match, jm_reading, best_distance)
        
        # Even without camera match, we can still use joulemeter data!
        # This is important - telemetry works even if camera fails
        synced = {
            'latitude': jm_lat,
            'longitude': jm_lon,
            'speed': jm_reading.get('speed', 0),
            'voltage': jm_reading['voltage'],
            'current': jm_reading['current'],
            'power': jm_reading['power'],
            'energy_wh': jm_reading['energy_wh'],
            'has_frame': False,
            'sync_distance_m': None,
            'synced_at': time.time()
        }
        self.synced_telemetry.append(synced)
        jm_reading['synced'] = True
        
        return synced
    
    def _create_synced_point(self, camera, joulemeter, sync_distance):
        """
        Merge camera and joulemeter data into a single synced point.
        """
        synced = {
            # Use camera GPS (usually mounted for better view)
            'latitude': camera['latitude'],
            'longitude': camera['longitude'],
            
            # Speed from whichever has it
            'speed': camera.get('speed') or joulemeter.get('speed', 0),
            
            # Joulemeter data
            'voltage': joulemeter['voltage'],
            'current': joulemeter['current'],
            'power': joulemeter['power'],
            'energy_wh': joulemeter['energy_wh'],
            
            # Camera data
            'frame_id': camera.get('frame_id'),
            'has_frame': camera.get('frame_id') is not None,
            
            # Sync metadata
            'sync_distance_m': sync_distance,
            'synced_at': time.time()
        }
        
        self.synced_telemetry.append(synced)
        self.stats['synced_count'] += 1
        self.stats['sync_distances'].append(sync_distance)
        
        # Mark both as used
        camera['synced'] = True
        joulemeter['synced'] = True
        
        return synced
    
    def get_sync_quality(self):
        """
        Report on how well the sync is working.
        """
        if not self.stats['sync_distances']:
            return {'status': 'No syncs yet'}
        
        distances = self.stats['sync_distances']
        return {
            'total_synced': self.stats['synced_count'],
            'avg_sync_distance_m': np.mean(distances),
            'max_sync_distance_m': np.max(distances),
            'min_sync_distance_m': np.min(distances),
            'sync_rate': self.stats['synced_count'] / max(1, self.stats['joulemeter_received'])
        }


# Create global data store
data_store = LocationSyncedDataStore(sync_threshold_meters=5.0)

print("✅ Location-synced data store created!")
print(f"   Sync threshold: {data_store.sync_threshold_m} meters")

## Part 4: Race State & Lap Detection

The race state tracks:
- Whether the race has started
- Current lap number
- Start/finish line position
- Data for each completed lap

In [ ]:
# ============================================
# 🏁 RACE STATE & LAP DETECTION
# ============================================

class RaceState:
    """
    Tracks the current state of the race and detects lap completions.
    
    Lap detection uses GPS proximity:
    1. Record starting position
    2. Wait until car leaves start area (>50m away)
    3. When car returns to start area (<30m), lap is complete!
    """
    
    def __init__(self):
        # Race status
        self.race_started = False
        self.current_lap = 0
        
        # Start/finish line position
        self.start_lat = None
        self.start_lon = None
        
        # Thresholds for lap detection
        self.start_threshold_m = 30   # Within 30m = at start line
        self.leave_threshold_m = 50   # Beyond 50m = left start area
        self.has_left_start = False
        
        # Lap data storage
        self.lap_start_idx = 0
        self.lap_data = {}  # {lap_num: {segments, efficiency, etc}}
        
        # Ideal lap (built from best segments)
        self.ideal_lap = None
        self.ideal_efficiency = 0
        self.ideal_racing_line = []  # Smoothed GPS path
        
        # Current position tracking
        self.current_lat = None
        self.current_lon = None
        self.current_heading = 0
    
    def process_telemetry(self, point):
        """
        Process a new telemetry point for lap detection.
        """
        lat = point.get('latitude')
        lon = point.get('longitude')
        
        if lat is None or lon is None:
            return None
        
        # Update current position
        if self.current_lat is not None:
            self.current_heading = calculate_bearing(
                self.current_lat, self.current_lon, lat, lon
            )
        self.current_lat = lat
        self.current_lon = lon
        
        # Set start position from first reading
        if self.start_lat is None:
            self.start_lat = lat
            self.start_lon = lon
            print(f"📍 Start position set: [{lat:.6f}, {lon:.6f}]")
            return 'start_set'
        
        # Check distance from start
        dist_from_start = haversine_meters(lat, lon, self.start_lat, self.start_lon)
        
        # Check if left start area
        if not self.has_left_start and dist_from_start > self.leave_threshold_m:
            self.has_left_start = True
            if not self.race_started:
                self.race_started = True
                self.current_lap = 1
                self.lap_start_idx = len(data_store.synced_telemetry) - 1
                print(f"\n🏁 RACE STARTED! Lap 1 begins")
                return 'race_started'
        
        # Check if returned to start (lap complete)
        if self.has_left_start and dist_from_start < self.start_threshold_m:
            self.has_left_start = False
            return 'lap_complete'
        
        return None


# Create global race state
race = RaceState()

print("✅ Race state initialized!")
print(f"   Lap detection: Leave > {race.leave_threshold_m}m, Return < {race.start_threshold_m}m")

## Part 5: Lap Segmentation & Efficiency Calculation

### Why Segment the Lap?

Different parts of the track have different characteristics:
- **Straight sections**: Higher speed, different energy profile
- **Corners**: Require different driving technique
- **Inclines/Declines**: Energy consumption varies

By segmenting, we can find the best approach for **each part** of the track!

In [ ]:
# ============================================
# 📊 LAP SEGMENTATION & EFFICIENCY
# ============================================

def segment_lap(telemetry_points, n_segments=4):
    """
    Divide a lap's telemetry into N equal segments.
    
    We segment by DISTANCE, not by number of points, because:
    - GPS sampling rate may vary
    - We want each segment to cover equal track length
    
    Returns: List of segment dictionaries with efficiency, path, etc.
    """
    if len(telemetry_points) < n_segments * 2:
        print(f"⚠️ Not enough points to segment ({len(telemetry_points)})")
        return None
    
    # Calculate cumulative distance for each point
    coords = [[p['latitude'], p['longitude']] for p in telemetry_points]
    cum_dist = cumulative_distance(coords)
    total_dist = cum_dist[-1]
    
    # Segment boundaries
    segment_length = total_dist / n_segments
    segment_names = ['Q1', 'Q2', 'Q3', 'Q4'][:n_segments]
    
    segments = []
    
    for i in range(n_segments):
        # Find points in this segment
        start_dist = i * segment_length
        end_dist = (i + 1) * segment_length
        
        # Get indices of points in this segment
        mask = (cum_dist >= start_dist) & (cum_dist < end_dist)
        if i == n_segments - 1:  # Last segment includes endpoint
            mask = (cum_dist >= start_dist)
        
        segment_points = [p for p, m in zip(telemetry_points, mask) if m]
        
        if len(segment_points) < 2:
            continue
        
        # Calculate segment metrics
        seg_coords = [[p['latitude'], p['longitude']] for p in segment_points]
        seg_distances = cumulative_distance(seg_coords)
        seg_distance_km = seg_distances[-1] if len(seg_distances) > 0 else 0
        
        # Energy used in this segment
        energy_values = [p.get('energy_wh', 0) for p in segment_points if p.get('energy_wh') is not None]
        if len(energy_values) >= 2:
            seg_energy_wh = max(energy_values) - min(energy_values)
        else:
            seg_energy_wh = 0.01  # Avoid division by zero
        
        # Efficiency: km per kWh
        seg_efficiency = seg_distance_km / (seg_energy_wh / 1000) if seg_energy_wh > 0 else 0
        
        # Average speed
        speeds = [p.get('speed', 0) for p in segment_points if p.get('speed') is not None]
        avg_speed = sum(speeds) / len(speeds) if speeds else 0
        
        segments.append({
            'segment_id': i,
            'segment_name': segment_names[i],
            'distance_km': seg_distance_km,
            'energy_wh': seg_energy_wh,
            'efficiency_km_kwh': seg_efficiency,
            'avg_speed': avg_speed,
            'point_count': len(segment_points),
            'path': seg_coords  # Raw GPS path for this segment
        })
    
    return segments


def complete_lap_processing(lap_num):
    """
    Process a completed lap: segment it, calculate efficiency, update ideal lap.
    """
    # Get telemetry for this lap
    lap_telemetry = data_store.synced_telemetry[race.lap_start_idx:]
    
    if len(lap_telemetry) < 20:
        print(f"⚠️ Lap {lap_num} too short ({len(lap_telemetry)} points)")
        race.current_lap += 1
        race.lap_start_idx = len(data_store.synced_telemetry)
        return
    
    print(f"\n{'='*60}")
    print(f"🏁 LAP {lap_num} COMPLETE!")
    print(f"{'='*60}")
    
    # Calculate total lap metrics
    coords = [[p['latitude'], p['longitude']] for p in lap_telemetry]
    total_distance = cumulative_distance(coords)[-1]
    
    energy_values = [p.get('energy_wh', 0) for p in lap_telemetry if p.get('energy_wh') is not None]
    total_energy = (max(energy_values) - min(energy_values)) if energy_values else 0.1
    
    lap_efficiency = total_distance / (total_energy / 1000) if total_energy > 0 else 0
    
    print(f"   📏 Distance: {total_distance*1000:.0f} m")
    print(f"   ⚡ Energy: {total_energy:.2f} Wh")
    print(f"   🏆 Efficiency: {lap_efficiency:.1f} km/kWh")
    
    # Segment the lap
    segments = segment_lap(lap_telemetry, n_segments=4)
    
    if segments:
        print(f"\n   Segments:")
        for seg in segments:
            print(f"      {seg['segment_name']}: {seg['efficiency_km_kwh']:.1f} km/kWh "
                  f"({seg['avg_speed']:.1f} km/h)")
    
    # Store lap data
    race.lap_data[lap_num] = {
        'total_distance_km': total_distance,
        'total_energy_wh': total_energy,
        'efficiency_km_kwh': lap_efficiency,
        'segments': {seg['segment_name']: seg for seg in segments} if segments else {},
        'raw_path': coords
    }
    
    # Update ideal lap
    update_ideal_lap()
    
    # Prepare for next lap
    race.current_lap += 1
    race.lap_start_idx = len(data_store.synced_telemetry)
    
    print(f"\n🔄 Ready for Lap {race.current_lap}")
    print(f"{'='*60}\n")


print("✅ Lap segmentation functions defined!")

## Part 6: Building the Ideal Lap

### The Core Algorithm

For each segment (Q1, Q2, Q3, Q4), we:
1. Look at that segment across ALL completed laps
2. Filter out outliers (unrealistic efficiency values)
3. Select the segment with the **highest efficiency**
4. Combine the selected segments into our "ideal lap"

The result is a lap that may have never actually been driven - it's the **theoretical best** based on real data!

In [ ]:
# ============================================
# 🏆 IDEAL LAP CONSTRUCTION
# ============================================

def update_ideal_lap():
    """
    Build the ideal lap by selecting the best segment from each quarter.
    
    This is called after every completed lap, so the ideal line
    IMPROVES as more laps are completed!
    """
    if not race.lap_data:
        return
    
    print(f"\n   🏆 UPDATING IDEAL LAP...")
    
    # Calculate bounds for outlier filtering
    # We use median + IQR to be robust against extreme values
    all_efficiencies = []
    for lap_info in race.lap_data.values():
        for seg in lap_info['segments'].values():
            eff = seg['efficiency_km_kwh']
            if 10 < eff < 1000:  # Basic sanity check
                all_efficiencies.append(eff)
    
    if not all_efficiencies:
        print("      ⚠️ No valid efficiency data")
        return
    
    median_eff = np.median(all_efficiencies)
    q1_eff = np.percentile(all_efficiencies, 25)
    q3_eff = np.percentile(all_efficiencies, 75)
    iqr = q3_eff - q1_eff
    
    # Outlier bounds: typically 1.5 * IQR, but we use 3 * IQR to be lenient
    min_valid = max(20, q1_eff - 3 * iqr)
    max_valid = min(500, q3_eff + 3 * iqr)
    
    print(f"      Efficiency filter: {min_valid:.0f} - {max_valid:.0f} km/kWh")
    
    # Build ideal lap
    race.ideal_lap = {}
    
    for q_name in ['Q1', 'Q2', 'Q3', 'Q4']:
        best_efficiency = 0
        best_lap = None
        
        for lap_num, lap_info in race.lap_data.items():
            if q_name not in lap_info['segments']:
                continue
            
            seg = lap_info['segments'][q_name]
            eff = seg['efficiency_km_kwh']
            
            # Check if within valid range AND better than current best
            if min_valid < eff < max_valid and eff > best_efficiency:
                best_efficiency = eff
                best_lap = lap_num
        
        if best_lap is not None:
            race.ideal_lap[q_name] = {
                'source_lap': best_lap,
                **race.lap_data[best_lap]['segments'][q_name]
            }
            print(f"      {q_name}: Lap {best_lap} → {best_efficiency:.1f} km/kWh")
        else:
            print(f"      {q_name}: No valid segment found")
    
    # Calculate ideal lap total efficiency
    if race.ideal_lap:
        total_dist = sum(s['distance_km'] for s in race.ideal_lap.values())
        total_energy = sum(s['energy_wh'] for s in race.ideal_lap.values())
        race.ideal_efficiency = total_dist / (total_energy / 1000) if total_energy > 0 else 0
        
        # Compare to best actual lap
        best_actual = max(l['efficiency_km_kwh'] for l in race.lap_data.values())
        improvement = ((race.ideal_efficiency / best_actual) - 1) * 100 if best_actual > 0 else 0
        
        print(f"\n      🏆 IDEAL LAP: {race.ideal_efficiency:.1f} km/kWh")
        print(f"      📊 Best actual: {best_actual:.1f} km/kWh")
        print(f"      📈 Potential improvement: {improvement:+.1f}%")
        
        # Build the racing line (will be smoothed in next step)
        build_racing_line()


print("✅ Ideal lap construction function defined!")

## Part 7: Racing Line Smoothing

### Why Smooth the Racing Line?

Raw GPS data has several problems:
1. **GPS Jitter**: Small random errors (±2-5 meters)
2. **Segment Boundaries**: Abrupt transitions between segments
3. **Variable Density**: Different sampling rates create uneven spacing

A smooth line is critical because:
- **Easier for driver to follow**: No sudden direction changes
- **More accurate representation**: Noise is filtered out
- **Better visualization**: Clean line on video overlay

### Smoothing Techniques

We use multiple techniques combined:

1. **Gaussian Smoothing**: Weighted average with neighbors
2. **Savitzky-Golay Filter**: Fits polynomials to preserve features
3. **Cubic Spline Interpolation**: Creates smooth curves between points
4. **Distance-Based Resampling**: Even spacing for consistent display

In [ ]:
# ============================================
# 🎯 RACING LINE SMOOTHING ALGORITHMS
# ============================================

def smooth_gaussian(coords, sigma=2):
    """
    Apply Gaussian smoothing to GPS coordinates.
    
    Gaussian smoothing averages each point with its neighbors,
    weighted by a bell curve. This removes high-frequency noise
    while preserving the overall shape.
    
    Parameters:
        coords: List of [lat, lon] pairs
        sigma: Standard deviation of Gaussian (higher = smoother)
    
    Returns: Smoothed coordinates
    """
    if len(coords) < 5:
        return coords
    
    coords = np.array(coords)
    
    # Apply Gaussian filter to lat and lon separately
    smoothed_lat = gaussian_filter1d(coords[:, 0], sigma=sigma)
    smoothed_lon = gaussian_filter1d(coords[:, 1], sigma=sigma)
    
    return list(zip(smoothed_lat, smoothed_lon))


def smooth_savgol(coords, window_length=11, polyorder=3):
    """
    Apply Savitzky-Golay filter to GPS coordinates.
    
    This filter fits a polynomial to a sliding window of points,
    then uses the polynomial's center value. It's excellent at
    preserving peaks and valleys while removing noise.
    
    Parameters:
        coords: List of [lat, lon] pairs
        window_length: Must be odd, larger = smoother
        polyorder: Polynomial degree (must be < window_length)
    
    Returns: Smoothed coordinates
    """
    if len(coords) < window_length:
        return coords
    
    coords = np.array(coords)
    
    # Ensure window_length is odd
    if window_length % 2 == 0:
        window_length += 1
    
    smoothed_lat = savgol_filter(coords[:, 0], window_length, polyorder)
    smoothed_lon = savgol_filter(coords[:, 1], window_length, polyorder)
    
    return list(zip(smoothed_lat, smoothed_lon))


def smooth_spline(coords, num_points=None, smoothing=0.0001):
    """
    Apply cubic spline interpolation to GPS coordinates.
    
    Splines create smooth curves that pass through (or near)
    control points. This gives the most natural-looking racing line.
    
    Parameters:
        coords: List of [lat, lon] pairs
        num_points: Number of output points (None = same as input)
        smoothing: Smoothing factor (0 = interpolate exactly, higher = smoother)
    
    Returns: Smoothed coordinates with even spacing
    """
    if len(coords) < 4:
        return coords
    
    coords = np.array(coords)
    
    # Calculate cumulative distance as parameter
    distances = cumulative_distance(coords.tolist())
    
    # Normalize to [0, 1]
    t = distances / distances[-1] if distances[-1] > 0 else np.linspace(0, 1, len(coords))
    
    # Create spline for lat and lon
    # We use UnivariateSpline for smoothing capability
    try:
        spline_lat = interpolate.UnivariateSpline(t, coords[:, 0], s=smoothing)
        spline_lon = interpolate.UnivariateSpline(t, coords[:, 1], s=smoothing)
    except:
        # Fallback to simple interpolation if spline fails
        return coords.tolist()
    
    # Generate evenly spaced points
    if num_points is None:
        num_points = len(coords)
    
    t_new = np.linspace(0, 1, num_points)
    
    smoothed_lat = spline_lat(t_new)
    smoothed_lon = spline_lon(t_new)
    
    return list(zip(smoothed_lat, smoothed_lon))


def resample_by_distance(coords, target_spacing_m=1.0):
    """
    Resample coordinates to have even spacing.
    
    This ensures the racing line has consistent point density,
    which makes visualization smoother and more predictable.
    
    Parameters:
        coords: List of [lat, lon] pairs
        target_spacing_m: Desired distance between points in meters
    
    Returns: Resampled coordinates with even spacing
    """
    if len(coords) < 2:
        return coords
    
    coords = np.array(coords)
    
    # Calculate total distance
    distances = cumulative_distance(coords.tolist())
    total_dist_m = distances[-1] * 1000  # Convert to meters
    
    if total_dist_m < target_spacing_m:
        return coords.tolist()
    
    # Number of output points
    num_points = int(total_dist_m / target_spacing_m)
    
    # Interpolate at even distances
    target_distances = np.linspace(0, distances[-1], num_points)
    
    resampled_lat = np.interp(target_distances, distances, coords[:, 0])
    resampled_lon = np.interp(target_distances, distances, coords[:, 1])
    
    return list(zip(resampled_lat, resampled_lon))


def smooth_segment_transitions(coords, segments_info, blend_distance_m=10):
    """
    Smooth the transitions between segments.
    
    When we combine segments from different laps, there can be
    discontinuities at the boundaries. This function blends
    the segments together for a seamless line.
    
    Parameters:
        coords: Combined coordinates from all segments
        segments_info: Info about where segments start/end
        blend_distance_m: Distance over which to blend
    
    Returns: Smoothed coordinates with seamless transitions
    """
    # For now, we use spline smoothing across the entire line
    # This naturally blends segment boundaries
    return smooth_spline(coords, smoothing=0.00001)


print("✅ Smoothing functions defined!")
print("   Available: Gaussian, Savitzky-Golay, Spline, Distance Resampling")

In [ ]:
# ============================================
# 🎯 COMPLETE RACING LINE BUILDER
# ============================================

def build_racing_line(target_spacing_m=1.0, smoothing_method='combined'):
    """
    Build the complete, smoothed ideal racing line.
    
    This combines all the smoothing techniques for maximum
    accuracy and visual smoothness.
    
    Parameters:
        target_spacing_m: Distance between points (1m = high detail)
        smoothing_method: 'gaussian', 'savgol', 'spline', or 'combined'
    
    The 'combined' method applies:
    1. Gaussian smoothing (remove GPS jitter)
    2. Spline interpolation (smooth curves)
    3. Distance resampling (even spacing)
    4. Final Savitzky-Golay pass (preserve features)
    """
    if not race.ideal_lap:
        print("⚠️ No ideal lap data to build racing line")
        return
    
    print("\n   🔄 Building smoothed racing line...")
    
    # Step 1: Combine paths from all segments
    raw_coords = []
    segment_boundaries = []  # Track where segments meet
    
    for q_name in ['Q1', 'Q2', 'Q3', 'Q4']:
        if q_name not in race.ideal_lap:
            continue
        
        seg_path = race.ideal_lap[q_name]['path']
        segment_boundaries.append(len(raw_coords))
        raw_coords.extend(seg_path)
    
    if len(raw_coords) < 10:
        print("      ⚠️ Not enough points for smoothing")
        race.ideal_racing_line = raw_coords
        return
    
    print(f"      Raw points: {len(raw_coords)}")
    
    # Step 2: Apply smoothing based on method
    if smoothing_method == 'gaussian':
        smoothed = smooth_gaussian(raw_coords, sigma=3)
        
    elif smoothing_method == 'savgol':
        smoothed = smooth_savgol(raw_coords, window_length=15, polyorder=3)
        
    elif smoothing_method == 'spline':
        smoothed = smooth_spline(raw_coords, smoothing=0.0001)
        
    elif smoothing_method == 'combined':
        # Multi-stage smoothing pipeline
        
        # Stage 1: Remove GPS jitter with Gaussian
        stage1 = smooth_gaussian(raw_coords, sigma=2)
        print(f"      After Gaussian: {len(stage1)} points")
        
        # Stage 2: Smooth curves with spline
        stage2 = smooth_spline(stage1, smoothing=0.00005)
        print(f"      After Spline: {len(stage2)} points")
        
        # Stage 3: Resample for even spacing
        stage3 = resample_by_distance(stage2, target_spacing_m)
        print(f"      After Resampling ({target_spacing_m}m): {len(stage3)} points")
        
        # Stage 4: Final polish with Savitzky-Golay
        if len(stage3) > 15:
            smoothed = smooth_savgol(stage3, window_length=11, polyorder=3)
        else:
            smoothed = stage3
        
        print(f"      Final smoothed: {len(smoothed)} points")
    
    else:
        smoothed = raw_coords
    
    # Store the racing line with segment info
    race.ideal_racing_line = []
    
    # Add segment info to each point
    total_points = len(smoothed)
    points_per_segment = total_points // 4
    segment_names = ['Q1', 'Q2', 'Q3', 'Q4']
    
    for i, (lat, lon) in enumerate(smoothed):
        seg_idx = min(i // points_per_segment, 3)
        seg_name = segment_names[seg_idx]
        
        # Get target speed from segment
        target_speed = 0
        if seg_name in race.ideal_lap:
            target_speed = race.ideal_lap[seg_name]['avg_speed']
        
        race.ideal_racing_line.append({
            'lat': lat,
            'lon': lon,
            'segment': seg_name,
            'target_speed': target_speed,
            'point_index': i
        })
    
    print(f"   ✅ Racing line built: {len(race.ideal_racing_line)} points")


print("✅ Racing line builder defined!")

## Part 8: Smoothness & Accuracy Testing

Now let's create tests to measure:
1. **Smoothness**: How "jerky" is the line? (Lower curvature change = smoother)
2. **Accuracy**: How close to the original data? (We don't want to smooth too much)
3. **Visual quality**: Can a driver follow it easily?

In [ ]:
# ============================================
# 📊 SMOOTHNESS & ACCURACY METRICS
# ============================================

def calculate_curvature(coords):
    """
    Calculate the curvature at each point of a path.
    
    Curvature measures how sharply the path is turning.
    Lower curvature = straighter line.
    
    Returns: Array of curvature values (units: 1/meters)
    """
    if len(coords) < 3:
        return np.array([0])
    
    coords = np.array(coords)
    
    # Convert to meters (approximate)
    # At equator, 1 degree ≈ 111km
    lat_center = np.mean(coords[:, 0])
    m_per_deg_lat = 111000
    m_per_deg_lon = 111000 * np.cos(np.radians(lat_center))
    
    x = coords[:, 1] * m_per_deg_lon
    y = coords[:, 0] * m_per_deg_lat
    
    # First and second derivatives
    dx = np.gradient(x)
    dy = np.gradient(y)
    ddx = np.gradient(dx)
    ddy = np.gradient(dy)
    
    # Curvature formula: |x'y'' - y'x''| / (x'^2 + y'^2)^(3/2)
    numerator = np.abs(dx * ddy - dy * ddx)
    denominator = (dx**2 + dy**2)**1.5
    
    # Avoid division by zero
    curvature = np.where(denominator > 1e-10, numerator / denominator, 0)
    
    return curvature


def calculate_curvature_change(coords):
    """
    Calculate the rate of change of curvature.
    
    This measures how "jerky" the path feels.
    Lower values = smoother transitions.
    
    Returns: Array of curvature change values
    """
    curvature = calculate_curvature(coords)
    return np.abs(np.gradient(curvature))


def calculate_heading_change(coords):
    """
    Calculate heading change between consecutive points.
    
    Large heading changes indicate sharp turns or GPS noise.
    
    Returns: Array of heading changes in degrees
    """
    if len(coords) < 3:
        return np.array([0])
    
    headings = []
    for i in range(1, len(coords)):
        h = calculate_bearing(
            coords[i-1][0], coords[i-1][1],
            coords[i][0], coords[i][1]
        )
        headings.append(h)
    
    # Calculate heading changes
    changes = []
    for i in range(1, len(headings)):
        diff = headings[i] - headings[i-1]
        # Normalize to -180 to 180
        if diff > 180:
            diff -= 360
        elif diff < -180:
            diff += 360
        changes.append(abs(diff))
    
    return np.array(changes)


def smoothness_score(coords):
    """
    Calculate an overall smoothness score (0-100).
    
    Higher score = smoother line.
    
    Based on:
    - Average curvature change (lower = better)
    - Maximum heading change (lower = better)
    - Standard deviation of heading changes (lower = better)
    """
    if len(coords) < 5:
        return 50  # Neutral score for insufficient data
    
    # Curvature change metric
    curv_change = calculate_curvature_change(coords)
    avg_curv_change = np.mean(curv_change)
    
    # Heading change metrics
    heading_changes = calculate_heading_change(coords)
    max_heading_change = np.max(heading_changes) if len(heading_changes) > 0 else 0
    std_heading_change = np.std(heading_changes) if len(heading_changes) > 0 else 0
    
    # Scoring (empirically tuned)
    # Curvature change score: 0 = perfect, >0.01 = poor
    curv_score = max(0, 100 - avg_curv_change * 10000)
    
    # Max heading change score: 0° = perfect, >30° = poor
    heading_score = max(0, 100 - max_heading_change * 3)
    
    # Std heading change score: consistency matters
    consistency_score = max(0, 100 - std_heading_change * 5)
    
    # Weighted average
    total_score = 0.4 * curv_score + 0.4 * heading_score + 0.2 * consistency_score
    
    return min(100, max(0, total_score))


def accuracy_score(original_coords, smoothed_coords):
    """
    Calculate how well the smoothed line represents the original.
    
    Higher score = closer to original data.
    
    We measure:
    - Average distance from smoothed points to nearest original point
    - Maximum deviation from original path
    """
    if len(original_coords) < 2 or len(smoothed_coords) < 2:
        return 50
    
    original = np.array(original_coords)
    smoothed = np.array(smoothed_coords)
    
    # For each smoothed point, find distance to nearest original point
    deviations = []
    for s_point in smoothed:
        min_dist = float('inf')
        for o_point in original:
            d = haversine_meters(s_point[0], s_point[1], o_point[0], o_point[1])
            if d < min_dist:
                min_dist = d
        deviations.append(min_dist)
    
    avg_deviation = np.mean(deviations)
    max_deviation = np.max(deviations)
    
    # Score: 0m deviation = 100, >5m average = 0
    avg_score = max(0, 100 - avg_deviation * 20)
    max_score = max(0, 100 - max_deviation * 10)
    
    return 0.7 * avg_score + 0.3 * max_score


def overall_quality_score(original_coords, smoothed_coords):
    """
    Calculate overall quality balancing smoothness and accuracy.
    
    Returns: Dictionary with all metrics
    """
    smooth_s = smoothness_score(smoothed_coords)
    acc_s = accuracy_score(original_coords, smoothed_coords)
    
    # Balance: We want both smoothness AND accuracy
    # Using harmonic mean to penalize low scores more
    if smooth_s > 0 and acc_s > 0:
        overall = 2 * smooth_s * acc_s / (smooth_s + acc_s)
    else:
        overall = 0
    
    # Additional metrics
    heading_changes = calculate_heading_change(smoothed_coords)
    
    return {
        'smoothness_score': round(smooth_s, 1),
        'accuracy_score': round(acc_s, 1),
        'overall_score': round(overall, 1),
        'max_heading_change_deg': round(np.max(heading_changes), 1) if len(heading_changes) > 0 else 0,
        'avg_heading_change_deg': round(np.mean(heading_changes), 2) if len(heading_changes) > 0 else 0,
        'point_count': len(smoothed_coords)
    }


print("✅ Quality metrics defined!")
print("   Metrics: Curvature, Heading Change, Smoothness Score, Accuracy Score")

## Part 9: Test with Sample Data

Let's create some test data and compare different smoothing methods.

In [ ]:
# ============================================
# 🧪 CREATE TEST DATA
# ============================================

def generate_test_track(center_lat=24.734833, center_lon=46.699108, 
                        radius_m=100, num_points=200, noise_m=2):
    """
    Generate a simulated oval track with GPS noise.
    
    This lets us test our smoothing algorithms with known ground truth.
    """
    # Create oval shape
    t = np.linspace(0, 2 * np.pi, num_points)
    
    # Oval with 2:1 aspect ratio
    x_m = radius_m * 2 * np.cos(t)
    y_m = radius_m * np.sin(t)
    
    # Add GPS noise
    x_m += np.random.normal(0, noise_m, num_points)
    y_m += np.random.normal(0, noise_m, num_points)
    
    # Convert to lat/lon
    m_per_deg_lat = 111000
    m_per_deg_lon = 111000 * np.cos(np.radians(center_lat))
    
    lats = center_lat + y_m / m_per_deg_lat
    lons = center_lon + x_m / m_per_deg_lon
    
    return list(zip(lats, lons))


# Generate test track
np.random.seed(33)  # Juju's favorite number!
test_track_noisy = generate_test_track(noise_m=3)  # 3 meters GPS noise
test_track_clean = generate_test_track(noise_m=0)  # Ground truth

print(f"✅ Generated test track: {len(test_track_noisy)} points")
print(f"   GPS noise: 3 meters")

In [ ]:
# ============================================
# 🧪 COMPARE SMOOTHING METHODS
# ============================================

print("\n" + "="*70)
print("🧪 SMOOTHING METHOD COMPARISON")
print("="*70)

methods = {
    'Raw (No Smoothing)': test_track_noisy,
    'Gaussian (σ=2)': smooth_gaussian(test_track_noisy, sigma=2),
    'Gaussian (σ=4)': smooth_gaussian(test_track_noisy, sigma=4),
    'Savitzky-Golay (w=11)': smooth_savgol(test_track_noisy, window_length=11),
    'Savitzky-Golay (w=21)': smooth_savgol(test_track_noisy, window_length=21),
    'Spline (s=0.0001)': smooth_spline(test_track_noisy, smoothing=0.0001),
    'Spline (s=0.001)': smooth_spline(test_track_noisy, smoothing=0.001),
}

# Combined method
stage1 = smooth_gaussian(test_track_noisy, sigma=2)
stage2 = smooth_spline(stage1, smoothing=0.00005)
stage3 = resample_by_distance(stage2, target_spacing_m=2)
if len(stage3) > 15:
    combined = smooth_savgol(stage3, window_length=11, polyorder=3)
else:
    combined = stage3
methods['Combined (Recommended)'] = combined

print(f"\n{'Method':<30} {'Smoothness':<12} {'Accuracy':<12} {'Overall':<12} {'Max Turn°':<10}")
print("-" * 76)

results = []
for name, coords in methods.items():
    scores = overall_quality_score(test_track_clean, coords)
    results.append((name, scores))
    print(f"{name:<30} {scores['smoothness_score']:<12} {scores['accuracy_score']:<12} "
          f"{scores['overall_score']:<12} {scores['max_heading_change_deg']:<10}")

# Find best method
best = max(results, key=lambda x: x[1]['overall_score'])
print(f"\n🏆 BEST METHOD: {best[0]}")
print(f"   Overall Score: {best[1]['overall_score']}")

In [ ]:
# ============================================
# 📊 VISUALIZE COMPARISON
# ============================================

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

# Ground truth for reference
clean = np.array(test_track_clean)

plot_methods = [
    ('Raw (Noisy)', test_track_noisy),
    ('Gaussian (σ=2)', methods['Gaussian (σ=2)']),
    ('Savitzky-Golay', methods['Savitzky-Golay (w=11)']),
    ('Spline', methods['Spline (s=0.0001)']),
    ('Combined', methods['Combined (Recommended)']),
    ('Ground Truth', test_track_clean)
]

for ax, (name, coords) in zip(axes, plot_methods):
    coords_arr = np.array(coords)
    
    # Plot ground truth in light gray
    ax.plot(clean[:, 1], clean[:, 0], 'lightgray', linewidth=3, label='Ground Truth')
    
    # Plot this method
    color = 'green' if name == 'Combined' else ('blue' if name == 'Ground Truth' else 'red')
    ax.plot(coords_arr[:, 1], coords_arr[:, 0], color, linewidth=1.5, label=name)
    
    # Score
    if name != 'Ground Truth':
        scores = overall_quality_score(test_track_clean, coords)
        ax.set_title(f"{name}\nScore: {scores['overall_score']:.0f} "
                     f"(S:{scores['smoothness_score']:.0f} A:{scores['accuracy_score']:.0f})")
    else:
        ax.set_title(f"{name}\n(Reference)")
    
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('smoothing_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Comparison saved to 'smoothing_comparison.png'")

In [ ]:
# ============================================
# 📊 HEADING CHANGE ANALYSIS
# ============================================

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

analyze_methods = [
    ('Raw (Noisy)', test_track_noisy, 'red'),
    ('Combined (Smoothed)', methods['Combined (Recommended)'], 'green'),
]

for (name, coords, color), (ax_heading, ax_curvature) in zip(
    analyze_methods, 
    [(axes[0, 0], axes[1, 0]), (axes[0, 1], axes[1, 1])]
):
    # Heading changes
    heading_changes = calculate_heading_change(coords)
    ax_heading.plot(heading_changes, color=color, linewidth=1)
    ax_heading.axhline(y=np.mean(heading_changes), color=color, linestyle='--', 
                       label=f'Mean: {np.mean(heading_changes):.1f}°')
    ax_heading.set_title(f'{name} - Heading Changes')
    ax_heading.set_xlabel('Point Index')
    ax_heading.set_ylabel('Heading Change (degrees)')
    ax_heading.legend()
    ax_heading.grid(True, alpha=0.3)
    
    # Curvature
    curvature = calculate_curvature(coords)
    ax_curvature.plot(curvature, color=color, linewidth=1)
    ax_curvature.set_title(f'{name} - Curvature')
    ax_curvature.set_xlabel('Point Index')
    ax_curvature.set_ylabel('Curvature (1/m)')
    ax_curvature.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('smoothness_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Analysis saved to 'smoothness_analysis.png'")
print("\n📊 KEY INSIGHT:")
print("   The smoothed line shows MUCH lower heading changes and curvature spikes.")
print("   This means the driver sees a steady, predictable path to follow!")

## Part 10: Live Streaming Server

Now let's create the server that receives data from both systems.

In [ ]:
# ============================================
# 🌐 FLASK SERVER FOR LIVE STREAMING
# ============================================

from flask import Flask, request, jsonify
import threading

app = Flask(__name__)

@app.route('/camera', methods=['POST'])
def receive_camera():
    """
    Receive data from camera system.
    
    Expected JSON:
    {
        "latitude": 24.734833,
        "longitude": 46.699108,
        "speed": 5.2,
        "frame_id": 1234
    }
    """
    data = request.json
    if not data:
        return jsonify({'error': 'No data'}), 400
    
    # Add to data store
    synced = data_store.add_camera_reading(
        lat=data.get('latitude'),
        lon=data.get('longitude'),
        speed=data.get('speed'),
        frame_id=data.get('frame_id')
    )
    
    # Process for lap detection
    if synced:
        event = race.process_telemetry(synced)
        if event == 'lap_complete':
            complete_lap_processing(race.current_lap)
    
    return jsonify({
        'status': 'ok',
        'synced': synced is not None,
        'race_started': race.race_started,
        'current_lap': race.current_lap,
        'ideal_available': len(race.ideal_racing_line) > 0
    })


@app.route('/joulemeter', methods=['POST'])
def receive_joulemeter():
    """
    Receive data from joulemeter system.
    
    Expected JSON:
    {
        "latitude": 24.734833,
        "longitude": 46.699108,
        "voltage": 48.5,
        "current": 2.3,
        "power": 111.55,
        "energy_wh": 12.5
    }
    """
    data = request.json
    if not data:
        return jsonify({'error': 'No data'}), 400
    
    synced = data_store.add_joulemeter_reading(
        lat=data.get('latitude'),
        lon=data.get('longitude'),
        voltage=data.get('voltage', 0),
        current=data.get('current', 0),
        power=data.get('power', 0),
        energy_wh=data.get('energy_wh', 0),
        speed=data.get('speed')
    )
    
    # Process for lap detection
    if synced:
        event = race.process_telemetry(synced)
        if event == 'lap_complete':
            complete_lap_processing(race.current_lap)
    
    return jsonify({
        'status': 'ok',
        'race_started': race.race_started,
        'current_lap': race.current_lap,
        'energy_wh': data.get('energy_wh')
    })


@app.route('/ideal_line', methods=['GET'])
def get_ideal_line():
    """
    Get the smoothed ideal racing line.
    
    Returns the GPS coordinates the driver should follow.
    """
    if not race.ideal_racing_line:
        return jsonify({
            'available': False,
            'message': 'Complete at least 1 lap',
            'current_lap': race.current_lap
        })
    
    # Calculate quality metrics
    coords = [[p['lat'], p['lon']] for p in race.ideal_racing_line]
    quality = overall_quality_score(
        coords,  # Using smoothed as reference for self-check
        coords
    )
    
    return jsonify({
        'available': True,
        'ideal_efficiency': round(race.ideal_efficiency, 1),
        'quality': quality,
        'point_count': len(race.ideal_racing_line),
        'racing_line': race.ideal_racing_line,
        'segments': {
            q: {
                'source_lap': race.ideal_lap[q]['source_lap'],
                'efficiency': round(race.ideal_lap[q]['efficiency_km_kwh'], 1),
                'target_speed': round(race.ideal_lap[q]['avg_speed'], 1)
            } for q in race.ideal_lap
        } if race.ideal_lap else {}
    })


@app.route('/status', methods=['GET'])
def get_status():
    """Get current system status."""
    return jsonify({
        'race_started': race.race_started,
        'current_lap': race.current_lap,
        'laps_completed': len(race.lap_data),
        'sync_quality': data_store.get_sync_quality(),
        'telemetry_points': len(data_store.synced_telemetry),
        'ideal_available': len(race.ideal_racing_line) > 0,
        'ideal_efficiency': round(race.ideal_efficiency, 1) if race.ideal_efficiency else None
    })


@app.route('/start', methods=['POST'])
def manual_start():
    """Manually start the race."""
    race.race_started = True
    race.current_lap = 1
    race.has_left_start = True
    race.lap_start_idx = len(data_store.synced_telemetry)
    print("🏁 RACE MANUALLY STARTED!")
    return jsonify({'status': 'started', 'lap': 1})


@app.route('/complete_lap', methods=['POST'])
def manual_complete():
    """Manually complete current lap."""
    complete_lap_processing(race.current_lap)
    return jsonify({'status': 'completed', 'new_lap': race.current_lap})


@app.route('/reset', methods=['POST'])
def reset():
    """Reset everything."""
    global data_store, race
    data_store = LocationSyncedDataStore()
    race = RaceState()
    print("🔄 SYSTEM RESET!")
    return jsonify({'status': 'reset'})


print("✅ Flask server defined!")

In [ ]:
# ============================================
# 🌐 START SERVER WITH NGROK (for Colab)
# ============================================

def run_server():
    """Run Flask server in background thread."""
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

# Start in background
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()
print("✅ Server running on port 5000")

# If in Colab, use ngrok for public URL
if IN_COLAB:
    try:
        from pyngrok import ngrok
        public_url = ngrok.connect(5000)
        print(f"\n🌐 PUBLIC URL: {public_url}")
        print(f"\n📡 ENDPOINTS:")
        print(f"   Camera:      POST {public_url}/camera")
        print(f"   Joulemeter:  POST {public_url}/joulemeter")
        print(f"   Ideal Line:  GET  {public_url}/ideal_line")
        print(f"   Status:      GET  {public_url}/status")
    except Exception as e:
        print(f"⚠️ ngrok not available: {e}")
        print("   Server running locally on port 5000")
else:
    print("\n📡 Server running on http://localhost:5000")

## Part 11: Simulate Live Data & Test Full System

In [ ]:
# ============================================
# 🧪 SIMULATE LIVE DATA STREAM
# ============================================

import requests

def simulate_race(num_laps=3, points_per_lap=100, server_url="http://localhost:5000"):
    """
    Simulate a race by sending data to both camera and joulemeter endpoints.
    
    This tests the complete system:
    - Location-based sync
    - Lap detection
    - Ideal lap building
    - Racing line smoothing
    """
    print("\n" + "="*70)
    print("🏎️ SIMULATING RACE")
    print("="*70)
    
    # Generate track
    center_lat = 24.734833
    center_lon = 46.699108
    
    # Reset system
    try:
        requests.post(f"{server_url}/reset")
    except:
        print("⚠️ Could not reset - continuing anyway")
    
    for lap in range(1, num_laps + 1):
        print(f"\n📍 Simulating Lap {lap}...")
        
        # Generate track with some variation per lap
        np.random.seed(33 + lap)  # Different noise per lap
        track = generate_test_track(
            center_lat=center_lat,
            center_lon=center_lon,
            radius_m=100,
            num_points=points_per_lap,
            noise_m=2 + lap * 0.5  # Slightly different each lap
        )
        
        # Simulate energy consumption with variation
        base_energy = 10  # Starting energy in Wh
        energy_per_point = 0.05 + np.random.uniform(-0.01, 0.01)  # Wh per point
        
        for i, (lat, lon) in enumerate(track):
            # Current energy
            energy = base_energy + i * energy_per_point
            
            # Simulate some efficiency variation by segment
            segment = i // (points_per_lap // 4)
            segment_efficiency_factor = 1 + (segment - 1.5) * 0.1 * np.random.uniform(0.5, 1.5)
            adjusted_energy = energy * segment_efficiency_factor
            
            # Send camera data
            camera_data = {
                'latitude': lat,
                'longitude': lon,
                'speed': 4 + np.random.uniform(0, 3),
                'frame_id': lap * 1000 + i
            }
            
            # Send joulemeter data (slightly offset position to test sync)
            offset_lat = lat + np.random.uniform(-0.00002, 0.00002)  # ~2m offset
            offset_lon = lon + np.random.uniform(-0.00002, 0.00002)
            
            joulemeter_data = {
                'latitude': offset_lat,
                'longitude': offset_lon,
                'voltage': 48 + np.random.uniform(-0.5, 0.5),
                'current': 2 + np.random.uniform(-0.3, 0.3),
                'power': 96 + np.random.uniform(-10, 10),
                'energy_wh': adjusted_energy
            }
            
            try:
                requests.post(f"{server_url}/camera", json=camera_data, timeout=1)
                requests.post(f"{server_url}/joulemeter", json=joulemeter_data, timeout=1)
            except:
                pass  # Server might be slow
            
            # Small delay to simulate real-time
            time.sleep(0.01)
        
        # Complete lap
        try:
            requests.post(f"{server_url}/complete_lap", timeout=2)
        except:
            pass
        
        print(f"   ✅ Lap {lap} data sent ({points_per_lap} points)")
    
    print("\n" + "="*70)
    print("🏁 SIMULATION COMPLETE")
    print("="*70)
    
    # Get final status
    try:
        status = requests.get(f"{server_url}/status", timeout=2).json()
        print(f"\n📊 Final Status:")
        print(f"   Laps completed: {status.get('laps_completed')}")
        print(f"   Telemetry points: {status.get('telemetry_points')}")
        print(f"   Ideal efficiency: {status.get('ideal_efficiency')} km/kWh")
        print(f"   Sync quality: {status.get('sync_quality')}")
    except Exception as e:
        print(f"⚠️ Could not get status: {e}")


print("✅ Race simulation function defined!")
print("   Run simulate_race() to test the full system")

In [ ]:
# Run the simulation!
simulate_race(num_laps=3, points_per_lap=100)

In [ ]:
# ============================================
# 📊 ANALYZE RESULTS
# ============================================

print("\n" + "="*70)
print("📊 RACING LINE QUALITY ANALYSIS")
print("="*70)

if race.ideal_racing_line:
    # Extract coordinates
    smoothed_coords = [[p['lat'], p['lon']] for p in race.ideal_racing_line]
    
    # Get raw coordinates for comparison
    raw_coords = []
    for q_name in ['Q1', 'Q2', 'Q3', 'Q4']:
        if q_name in race.ideal_lap:
            raw_coords.extend(race.ideal_lap[q_name]['path'])
    
    # Calculate quality
    quality = overall_quality_score(raw_coords, smoothed_coords)
    
    print(f"\n🎯 QUALITY SCORES:")
    print(f"   Smoothness:  {quality['smoothness_score']}/100 "
          f"({'Excellent' if quality['smoothness_score'] > 80 else 'Good' if quality['smoothness_score'] > 60 else 'Needs Improvement'})")
    print(f"   Accuracy:    {quality['accuracy_score']}/100 "
          f"({'Excellent' if quality['accuracy_score'] > 80 else 'Good' if quality['accuracy_score'] > 60 else 'Needs Improvement'})")
    print(f"   Overall:     {quality['overall_score']}/100")
    print(f"\n📐 METRICS:")
    print(f"   Max heading change:  {quality['max_heading_change_deg']}°")
    print(f"   Avg heading change:  {quality['avg_heading_change_deg']}°")
    print(f"   Total points:        {quality['point_count']}")
    
    # Recommendations
    print(f"\n💡 RECOMMENDATIONS:")
    if quality['max_heading_change_deg'] > 20:
        print("   - Consider increasing smoothing for sharp corners")
    if quality['accuracy_score'] < 70:
        print("   - Reduce smoothing to stay closer to actual path")
    if quality['smoothness_score'] > 85 and quality['accuracy_score'] > 85:
        print("   - ✅ Racing line is well-optimized!")
else:
    print("\n⚠️ No racing line available yet. Complete at least one lap.")

In [ ]:
# ============================================
# 🗺️ VISUALIZE IDEAL RACING LINE
# ============================================

if race.ideal_racing_line:
    # Create map
    coords = [[p['lat'], p['lon']] for p in race.ideal_racing_line]
    center_lat = np.mean([c[0] for c in coords])
    center_lon = np.mean([c[1] for c in coords])
    
    m = folium.Map(location=[center_lat, center_lon], zoom_start=17)
    
    # Color by segment
    segment_colors = {'Q1': 'red', 'Q2': 'blue', 'Q3': 'green', 'Q4': 'orange'}
    
    # Group points by segment
    current_segment = None
    segment_coords = []
    
    for point in race.ideal_racing_line:
        if point['segment'] != current_segment:
            if segment_coords and current_segment:
                # Draw previous segment
                color = segment_colors.get(current_segment, 'purple')
                folium.PolyLine(
                    segment_coords,
                    weight=5,
                    color=color,
                    opacity=0.8,
                    popup=f"{current_segment}: {race.ideal_lap[current_segment]['efficiency_km_kwh']:.1f} km/kWh"
                ).add_to(m)
            
            current_segment = point['segment']
            segment_coords = []
        
        segment_coords.append([point['lat'], point['lon']])
    
    # Draw last segment
    if segment_coords and current_segment:
        color = segment_colors.get(current_segment, 'purple')
        folium.PolyLine(segment_coords, weight=5, color=color, opacity=0.8).add_to(m)
    
    # Add start marker
    if race.start_lat:
        folium.Marker(
            [race.start_lat, race.start_lon],
            popup='Start/Finish',
            icon=folium.Icon(color='green', icon='flag')
        ).add_to(m)
    
    # Add legend
    legend_html = '''
    <div style="position: fixed; bottom: 50px; left: 50px; z-index: 1000;
                background-color: white; padding: 10px; border-radius: 5px;
                border: 2px solid gray;">
        <b>Ideal Racing Line</b><br>
        <span style="color: red;">■</span> Q1<br>
        <span style="color: blue;">■</span> Q2<br>
        <span style="color: green;">■</span> Q3<br>
        <span style="color: orange;">■</span> Q4
    </div>
    '''
    m.get_root().html.add_child(folium.Element(legend_html))
    
    # Save and display
    m.save('ideal_racing_line_smooth.html')
    print("✅ Map saved to 'ideal_racing_line_smooth.html'")
    
    # Display in notebook
    display(m)
else:
    print("⚠️ No racing line to display")

---

## 📚 Summary: How the Ideal Racing Line is Created

### Step 1: Data Collection
- **Camera system** sends: GPS position + frame ID
- **Joulemeter system** sends: GPS position + energy data
- **Location sync**: Match readings within 5 meters

### Step 2: Lap Detection
- Start position recorded from first GPS reading
- Lap completes when car returns within 30m of start
- Each lap is segmented into 4 quarters (Q1-Q4)

### Step 3: Efficiency Calculation
- For each segment: `Efficiency = Distance / (Energy / 1000)` km/kWh
- Outliers filtered using IQR method

### Step 4: Ideal Lap Construction
- For each quarter (Q1, Q2, Q3, Q4):
  - Look at that quarter across ALL completed laps
  - Select the one with highest efficiency
- Result: A "best of the best" composite lap

### Step 5: Smoothing Pipeline
1. **Gaussian filter**: Remove GPS jitter (σ=2)
2. **Spline interpolation**: Create smooth curves (s=0.00005)
3. **Distance resampling**: Even point spacing (1-2 meters)
4. **Savitzky-Golay**: Final polish while preserving features

### Step 6: Quality Assurance
- **Smoothness score**: Based on curvature and heading changes
- **Accuracy score**: Distance from original data points
- **Overall score**: Harmonic mean of both

---

## 🎯 Optimization Tips

| Parameter | Effect | When to Adjust |
|-----------|--------|----------------|
| Gaussian σ | Higher = smoother | Increase if GPS is noisy |
| Spline smoothing | Higher = smoother | Increase for tighter tracks |
| Resample spacing | Lower = more detail | Decrease for precise display |
| Savgol window | Larger = smoother | Increase for long straights |
| Sync threshold | Higher = more matches | Increase if GPS is inaccurate |

---

## 🏎️ Good luck at Shell Eco-marathon Qatar 2026!

In [ ]:
# ============================================
# 📥 DOWNLOAD ALL FILES
# ============================================

if IN_COLAB:
    # Download map
    if 'ideal_racing_line_smooth.html' in os.listdir():
        files.download('ideal_racing_line_smooth.html')
    
    # Export racing line as JSON
    if race.ideal_racing_line:
        export_data = {
            'metadata': {
                'created': datetime.now().isoformat(),
                'ideal_efficiency': round(race.ideal_efficiency, 2),
                'smoothing_method': 'combined',
                'point_count': len(race.ideal_racing_line)
            },
            'segments': {
                q: {
                    'source_lap': race.ideal_lap[q]['source_lap'],
                    'efficiency': round(race.ideal_lap[q]['efficiency_km_kwh'], 2),
                    'target_speed': round(race.ideal_lap[q]['avg_speed'], 1)
                } for q in race.ideal_lap
            } if race.ideal_lap else {},
            'racing_line': race.ideal_racing_line
        }
        
        with open('ideal_racing_line_v7.json', 'w') as f:
            json.dump(export_data, f, indent=2)
        
        files.download('ideal_racing_line_v7.json')
        print("✅ Files downloaded!")
    else:
        print("⚠️ No racing line to export")
else:
    print("💾 Files saved locally")